---
format: 
  html:
    embed-resources: true
execute:
  echo: true
code-fold: true
title: Lab 5: Insurance Costs
author: James Compagno
jupyter: python3
---

In [180]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [181]:
lr = LinearRegression()


ames = pd.read_csv("AmesHousing.csv")
X = ames[["Gr Liv Area", "TotRms AbvGrd"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_s = (X_train - X_train.mean())/X_train.std()

lr_fitted = lr.fit(X_train_s, y_train)
lr_fitted.coef_

array([ 72829.84009875, -19320.19793351])

In [182]:
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [183]:
new_house = pd.DataFrame(data = {"Gr Liv Area": [889], "TotRms AbvGrd": [6]})
new_house

,Gr Liv Area,TotRms AbvGrd
0,889,6


In [184]:
new_house_s = (new_house - new_house.mean())/new_house.std()
new_house_s

,Gr Liv Area,TotRms AbvGrd
0,NaN,NaN


In [185]:
X_test_s = (X_test - X_train.mean())/X_train.std()
y_preds = lr_fitted.predict(X_test_s)

r2_score(y_test, y_preds)

0.5136474646729811

In [186]:
new_house_s = (new_house - X_train.mean())/X_train.std()
lr_fitted.predict(new_house_s)

array([98361.41516052])

In [187]:
lr_pipeline = Pipeline(
  [("standardize", StandardScaler()),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('standardize', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [188]:
lr_pipeline_fitted = lr_pipeline.fit(X_train, y_train)

y_preds = lr_pipeline_fitted.predict(X_test)
r2_score(y_test, y_preds)

0.5136474646729811

In [189]:
lr_pipeline_fitted.predict(new_house)

array([98361.41516052])

In [190]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)

lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [191]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

lr_fitted = lr_pipeline.fit(X_train, y_train)

In [192]:
ct_fitted = ct.fit(X_train)

ct.transform(X_train)

array([[ 1.        ,  0.        ,  0.        , ...,  0.        , -0.03593684,
        -0.28287775],
       [ 1.        ,  0.        ,  0.        , ...,  0.        , -0.06165322,
        -0.28287775],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  1.42000579,
         0.98804745],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,  0.42893615,
         1.62351006],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  1.7780569 ,
         0.98804745],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.12825234,
         0.35258485]], shape=(2197, 7))

In [193]:
ct.transform(X_test)

array([[ 1.        ,  0.        ,  0.        , ...,  0.        , -0.08539141,
         0.35258485],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.69796902,
         0.35258485],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.94326371,
         0.35258485],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        , -0.0062641 ,
         0.35258485],
       [ 1.        ,  0.        ,  0.        , ...,  0.        , -0.16451873,
         0.35258485],
       [ 1.        ,  0.        ,  0.        , ...,  0.        , -1.0823956 ,
        -1.55380296]], shape=(733, 7))

In [194]:
lr_pipeline_fitted.named_steps['linear_regression'].coef_

array([ 72813.26337381, -19315.80048314])

In [195]:
type(y_preds)

numpy.ndarray

In [196]:
lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")


ct.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,standardize__Gr Liv Area,standardize__TotRms AbvGrd
361,1.0,0.0,0.0,0.0,0.0,-0.035937,-0.282878
822,1.0,0.0,0.0,0.0,0.0,-0.061653,-0.282878
1614,1.0,0.0,0.0,0.0,0.0,1.420006,0.988047
258,1.0,0.0,0.0,0.0,0.0,0.361678,0.352585
2202,1.0,0.0,0.0,0.0,0.0,-0.124955,0.352585
...,...,...,...,...,...,...,...
1990,1.0,0.0,0.0,0.0,0.0,0.282551,1.623510
1744,1.0,0.0,0.0,0.0,0.0,-0.461246,-0.282878
1515,0.0,1.0,0.0,0.0,0.0,0.428936,1.623510
1769,1.0,0.0,0.0,0.0,0.0,1.778057,0.988047


In [197]:
ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

ct_inter.fit_transform(X_train)

,interaction__1,interaction__Gr Liv Area,interaction__TotRms AbvGrd,interaction__Gr Liv Area TotRms AbvGrd
361,1.0,1481.0,6.0,8886.0
822,1.0,1468.0,6.0,8808.0
1614,1.0,2217.0,8.0,17736.0
258,1.0,1682.0,7.0,11774.0
2202,1.0,1436.0,7.0,10052.0
...,...,...,...,...
1990,1.0,1642.0,9.0,14778.0
1744,1.0,1266.0,6.0,7596.0
1515,1.0,1716.0,9.0,15444.0
1769,1.0,2398.0,8.0,19184.0


In [198]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__TotRms AbvGrd", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct_dummies.fit_transform(X_train)
X_train_dummified

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,remainder__Order,remainder__PID,remainder__MS SubClass,remainder__MS Zoning,remainder__Lot Frontage,remainder__Lot Area,remainder__Street,remainder__Alley,remainder__Lot Shape,remainder__Land Contour,remainder__Utilities,remainder__Lot Config,remainder__Land Slope,remainder__Neighborhood,remainder__Condition 1,remainder__Condition 2,remainder__House Style,remainder__Overall Qual,remainder__Overall Cond,remainder__Year Built,remainder__Year Remod/Add,remainder__Roof Style,remainder__Roof Matl,remainder__Exterior 1st,remainder__Exterior 2nd,remainder__Mas Vnr Type,remainder__Mas Vnr Area,remainder__Exter Qual,remainder__Exter Cond,remainder__Foundation,remainder__Bsmt Qual,remainder__Bsmt Cond,remainder__Bsmt Exposure,remainder__BsmtFin Type 1,remainder__BsmtFin SF 1,...,remainder__Heating QC,remainder__Central Air,remainder__Electrical,remainder__1st Flr SF,remainder__2nd Flr SF,remainder__Low Qual Fin SF,remainder__Gr Liv Area,remainder__Bsmt Full Bath,remainder__Bsmt Half Bath,remainder__Full Bath,remainder__Half Bath,remainder__Bedroom AbvGr,remainder__Kitchen AbvGr,remainder__Kitchen Qual,remainder__TotRms AbvGrd,remainder__Functional,remainder__Fireplaces,remainder__Fireplace Qu,remainder__Garage Type,remainder__Garage Yr Blt,remainder__Garage Finish,remainder__Garage Cars,remainder__Garage Area,remainder__Garage Qual,remainder__Garage Cond,remainder__Paved Drive,remainder__Wood Deck SF,remainder__Open Porch SF,remainder__Enclosed Porch,remainder__3Ssn Porch,remainder__Screen Porch,remainder__Pool Area,remainder__Pool QC,remainder__Fence,remainder__Misc Feature,remainder__Misc Val,remainder__Mo Sold,remainder__Yr Sold,remainder__Sale Type,remainder__Sale Condition
361,1.0,0.0,0.0,0.0,0.0,362,527164090,60,RL,65.0,8125,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,2Story,6,5,1994,1995,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,...,Gd,Y,SBrkr,702,779,0,1481,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1994.0,Fin,2.0,343.0,TA,TA,Y,0,36,0,0,0,0,NaN,NaN,NaN,0,3,2009,WD,Normal
822,1.0,0.0,0.0,0.0,0.0,823,906378110,20,RL,71.0,9178,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Story,8,5,2008,2008,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,Gd,TA,PConc,Gd,TA,Av,Unf,0.0,...,Ex,Y,SBrkr,1468,0,0,1468,0.0,0.0,2,0,3,1,Gd,6,Typ,1,Gd,Attchd,2008.0,RFn,3.0,904.0,TA,TA,Y,192,142,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
1614,1.0,0.0,0.0,0.0,0.0,1615,526355190,20,RL,95.0,15865,Pave,NaN,IR1,Lvl,AllPub,Inside,Mod,NAmes,Norm,Norm,1Story,8,6,1970,1970,Flat,Tar&Grv,Wd Sdng,Wd Sdng,NaN,0.0,Gd,Gd,PConc,TA,Gd,Gd,ALQ,351.0,...,Ex,Y,SBrkr,2217,0,0,2217,1.0,0.0,2,0,4,1,Gd,8,Typ,1,TA,Attchd,1970.0,Unf,2.0,621.0,TA,TA,Y,81,207,0,0,224,0,NaN,NaN,NaN,0,10,2007,WD,Normal
258,1.0,0.0,0.0,0.0,0.0,259,907185060,60,RL,70.0,8750,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,2Story,6,5,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,116.0,TA,TA,PConc,Gd,TA,No,GLQ,505.0,...,Ex,Y,SBrkr,804,878,0,1682,0.0,0.0,2,1,3,1,Gd,7,Typ,0,NaN,Attchd,1998.0,RFn,2.0,523.0,TA,TA,Y,0,77,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
2202,1.0,0.0,0.0,0.0,0.0,2203,909252170,70,RL,53.0,6720,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,2Story,6,7,1921,1950,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,...,TA,N,SBrkr,851,585,0,1436,0.0,0.0,1,0,3,1,TA,7,Typ,1,Gd,Detchd,1992.0,Unf,1.0,228.0,TA,TA,Y,184,0,0,0,0,0,NaN,NaN,NaN,0,4,2007,CWD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,1.0,0.0,0.0,0.0,0.0,1991,902300040,70,RM,53.0,9863,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2Story,6,6,1927,1950,Gambrel,CompShg,Wd 

In [199]:
ct_inter.fit_transform(X_train_dummified)

,interaction__1,interaction__remainder__TotRms AbvGrd,interaction__dummify__Bldg Type_1Fam,interaction__remainder__TotRms AbvGrd dummify__Bldg Type_1Fam
361,1.0,6.0,1.0,6.0
822,1.0,6.0,1.0,6.0
1614,1.0,8.0,1.0,8.0
258,1.0,7.0,1.0,7.0
2202,1.0,7.0,1.0,7.0
...,...,...,...,...
1990,1.0,9.0,1.0,9.0
1744,1.0,6.0,1.0,6.0
1515,1.0,9.0,0.0,0.0
1769,1.0,8.0,1.0,8.0


Make Your own Size and Number of Rooms

In [200]:
model_library = {}
records = []

In [201]:
ct_1 = ColumnTransformer(
  [
    ("select", "passthrough", ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop")

In [202]:
lr_pipeline = Pipeline(
  [("select", ct_1),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('select',
                 ColumnTransformer(transformers=[('select', 'passthrough',
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [203]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

lr_fitted = lr_pipeline.fit(X_train, y_train)
model_library["LivArea_Rooms"] = lr_fitted 

In [204]:
y_test_pred = model_library["LivArea_Rooms"].predict(X_test)
mse = mean_squared_error(y_test, y_test_pred) 
records.append({
    "Model": "LivArea_Rooms", "Split": "Test",
    "RMSE": np.sqrt(mse), "MSE": mse, "R2": r2_score(y_test, y_test_pred)  
})

In [205]:
cumulative_models = (pd.DataFrame(records))
cumulative_models

,Model,Split,RMSE,MSE,R2
0,LivArea_Rooms,Test,55373.843704,3.066263e+09,0.524547


# Using size, number of rooms, and building type.

In [206]:
preprocess = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("num", "passthrough", ["Gr Liv Area", "TotRms AbvGrd"]),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

lr_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("linear_regression", LinearRegression())
])

lr_fitted = lr_pipeline.fit(X_train, y_train)
model_library["LivArea_Rooms_BlgdType"] = lr_fitted 

In [208]:
y_test_pred = model_library["LivArea_Rooms_BlgdType"].predict(X_test)
mse = mean_squared_error(y_test, y_test_pred) 
records.append({
    "Model": "LivArea_Rooms_BlgdType", "Split": "Test",
    "RMSE": np.sqrt(mse), "MSE": mse, "R2": r2_score(y_test, y_test_pred)  
})

In [209]:
cumulative_models = (pd.DataFrame(records))
cumulative_models

,Model,Split,RMSE,MSE,R2
0,LivArea_Rooms,Test,55373.843704,3.066263e+09,0.524547
1,LivArea_Rooms_BlgdType,Test,53380.249826,2.849451e+09,0.558166


# Using size and building type, and their interaction.

In [ ]:
preprocess = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("num", "passthrough", ["Gr Liv Area", "TotRms AbvGrd"]),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

interaction 

In [ ]:
lr_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("linear_regression", LinearRegression())
])

In [ ]:
lr_fitted = lr_pipeline.fit(X_train, y_train)
model_library["LivArea_Rooms_BlgdType"] = lr_fitted 

In [ ]:
y_test_pred = model_library["LivArea_Rooms_BlgdType"].predict(X_test)
mse = mean_squared_error(y_test, y_test_pred) 
records.append({
    "Model": "LivArea_Rooms_BlgdType", "Split": "Test",
    "RMSE": np.sqrt(mse), "MSE": mse, "R2": r2_score(y_test, y_test_pred)  
})

cumulative_models = (pd.DataFrame(records))
cumulative_models

# MISSING WORK

Once again consider four modeling options for house price:

    -Using only the size and number of rooms.
    -Using size, number of rooms, and building type.
    -Using size and building type, and their interaction.
    -Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
    
Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?